# Introduction to basic CUDA code generation

This notebook demonstrates CUDA code generation for some basic neuron and synapse models.

In [1]:
import inspect
import sys

from neural.model.synapse import *
from neural.model.neuron import *
from neural.codegen.cuda import CudaKernelGenerator
from neural.codegen.neurodriver import NeuroDriverKernelGenerator

In [2]:
from neurokernel.LPU.NDComponents.NDComponent import NDComponent

## Hodgkin-Huxley Neuron

In [3]:
hh = HodgkinHuxley()
print(inspect.getsource(hh.ode))

    def ode(self, stimulus=0.):

        alpha = np.exp(-(self.v+55.)/10.)-1.
        beta = (0.125*np.exp(-(self.v+65.)/80.))
        if abs(alpha) <= 1e-7:
            self.d_n = 0.1 * (1.-self.n) - beta * self.n
        else:
            self.d_n = (-0.01*(self.v+55.)/alpha) * (1.-self.n) - beta * self.n

        alpha = np.exp(-(self.v+40.)/10.)-1.
        beta = (4.*np.exp(-(self.v+65.)/18.))
        if abs(alpha) <= 1e-7:
            self.d_m = (1.-self.m) - beta * self.m
        else:
            self.d_m = (-0.1*(self.v+40.)/alpha) * (1.-self.m) - beta * self.m

        alpha = (0.07 * np.exp(-(self.v+65.)/20.))
        beta = 1. / (np.exp(-(self.v+35.)/10.)+1.)
        self.d_h = alpha * (1-self.h) - beta * self.h

        i_na = self.gNa * np.power(self.m, 3) * self.h * (self.v - self.ENa)
        i_k = self.gK * np.power(self.n, 4) * (self.v - self.EK)
        i_l = self.gL * (self.v - self.EL)

        self.d_v = stimulus - i_na - i_k - i_l



#### NeuroDriver Code

In [4]:
nd_code_generator = NeuroDriverKernelGenerator(hh, outputs='v')
nd_code_generator.generate()
print(nd_code_generator.cuda_src)



#define  V_MIN		-80
#define  V_MAX		30
#define  N_MIN		0.0
#define  N_MAX		1.0
#define  M_MIN		0.0
#define  M_MAX		1.0
#define  H_MIN		0.0
#define  H_MAX		1.0

struct States {
    float v;
    float n;
    float m;
    float h;
};

struct Derivatives {
    float v;
    float n;
    float m;
    float h;
};


__device__ void clip(States &states)
{
    states.v = fmaxf(states.v, V_MIN);
    states.v = fminf(states.v, V_MAX);
    states.n = fmaxf(states.n, N_MIN);
    states.n = fminf(states.n, N_MAX);
    states.m = fmaxf(states.m, M_MIN);
    states.m = fminf(states.m, M_MAX);
    states.h = fmaxf(states.h, H_MIN);
    states.h = fminf(states.h, H_MAX);
}

__device__ void forward(
    States &states,
    Derivatives &gstates,
    float dt
)
{
    states.v += dt * gstates.v;
    states.n += dt * gstates.n;
    states.m += dt * gstates.m;
    states.h += dt * gstates.h;
}

__device__ int ode(
    States &states,
    Derivatives &gstates,
    float EK,
    float GNA,
    float ENA,
    f

```
ipdb>  params_dict
{'EL': array([-17.]), 'refperiod': array([1.]), 'gL': array([0.3]), 'ms': array([-5.3]), 'Ea': array([-75.]), 'hs': array([-12.]), 'EK': array([-72.]), 'ENa': array([55.]), 'gNa': array([120.]), 'sigma': array([2.05]), 'ns': array([-4.3]), 'ga': array([47.7]), 'gK': array([20.]), 'pre': {'I': array([0], dtype=int32)}, 'cumpre': {'I': array([0, 1], dtype=int32)}, 'npre': {'I': array([1], dtype=int32)}, 'conn_data': {'I': {'delay': array([0], dtype=int32)}}}m
```

```
ipdb>  access_buffers
{'I': <neurokernel.LPU.MemoryManager.CircularArray object at 0x7fb888fea278>}
```

```
ipdb>  update_pointers
{'spike_state': 140429653704712, 'V': 140429653705224}
```

In [7]:
from collections import OrderedDict

class NeuralNDComponent(NDComponent):
    accesses = None  # list
    updates = None   # list
    params = None    # list
    internals = None # orderedDict
    time_scale = 1.  # scales dt
    _cuda_src = None # cuda source code
    _has_rand = False
    
    def maximum_dt_allowed(self):
        return 1e-5

    def __init__(
        self,
        params_dict,
        access_buffers,
        dt,
        LPU_id=None,
        debug=False,
        cuda_verbose=False,
    ):
        if cuda_verbose:
            self.compile_options = ["--ptxas-options=-v", "--expt-relaxed-constexpr"]
        else:
            self.compile_options = ["--expt-relaxed-constexpr"]

        self.debug = debug
        self.LPU_id = LPU_id
        self.num_comps = params_dict[self.params[0]].size
        self.dtype = params_dict[self.params[0]].dtype

        self.dt = dt * self.time_scale
        self.params_dict = params_dict
        self.access_buffers = access_buffers

        self.internal_states = {
            c: garray.zeros(self.num_comps, dtype=self.dtype) + self.internals[c]
            for c in self.internals
        }

        self.inputs = {
            k: garray.empty(self.num_comps, dtype=self.access_buffers[k].dtype)
            for k in self.accesses
        }

        dtypes = {"dt": self.dtype}
        dtypes.update({"state_" + k: self.internal_states[k].dtype for k in self.internals})
        dtypes.update({"param_" + k: self.params_dict[k].dtype for k in self.params})
        dtypes.update({"input_" + k.format(k): self.inputs[k].dtype for k in self.accesses})
        dtypes.update({"output_" + k: self.dtype for k in self.updates})
        self.update_func = self.get_update_func(dtypes)

        if self._has_rand:
            import neurokernel.LPU.utils.curand as curand
            self.randState = curand.curand_setup(self.num_comps, np.random.randint(10000))

    def run_step(self, update_pointers, st=None):
        for k in self.inputs:
            self.sum_in_variable(k, self.inputs[k], st=st)

        args = [self.internal_states[k].gpudata for k in self.internals] \
            + [self.params_dict[k].gpudata for k in self.params] \
            + [self.inputs[k].gpudata for k in self.accesses] \
            + [update_pointers[k] for k in self.updates]
        if self._has_rand:
            args += [self.randState.gpudata]

        self.update_func.prepared_async_call(
            self.update_func.grid,
            self.update_func.block,
            st,
            self.num_comps,
            self.dt,
            *args
        )


    def get_update_func(self, dtypes):
        assert self._cuda_src is not None, "_cuda_src is None, cannot compile"
                
        from pycuda.compiler import SourceModule

        mod = SourceModule(
            self._cuda_src,
            options=self.compile_options,
            no_extern_c=True,
        )
        func = mod.get_function(self.__class__.__name__)
        type_dict = {k: dtype_to_ctype(dtypes[k]) for k in dtypes}

        func.prepare("i" + np.dtype(self.dtype).char + "P" + "P" * (len(type_dict) - 1))
        func.block = (256, 1, 1)
        func.grid = (
            min(6 * cuda.Context.get_device().MULTIPROCESSOR_COUNT,
                (self.num_comps - 1) // 256 + 1),
            1,
        )
        return func

#### CUDA Code

In [3]:
code_generator = CudaKernelGenerator(hh)
code_generator.generate()
print(code_generator.cuda_src)



#define  GNA		120.0
#define  GK		36.0
#define  GL		0.3
#define  ENA		50.0
#define  EK		-77.0
#define  EL		-54.387

#define  V_MIN		-80
#define  V_MAX		30
#define  N_MIN		0.0
#define  N_MAX		1.0
#define  M_MIN		0.0
#define  M_MAX		1.0
#define  H_MIN		0.0
#define  H_MAX		1.0

struct States {
    float v;
    float n;
    float m;
    float h;
};

struct Derivatives {
    float v;
    float n;
    float m;
    float h;
};


__device__ void clip(States &states)
{
    states.v = fmaxf(states.v, V_MIN);
    states.v = fminf(states.v, V_MAX);
    states.n = fmaxf(states.n, N_MIN);
    states.n = fminf(states.n, N_MAX);
    states.m = fmaxf(states.m, M_MIN);
    states.m = fminf(states.m, M_MAX);
    states.h = fmaxf(states.h, H_MIN);
    states.h = fminf(states.h, H_MAX);
}

__device__ void forward(
    States &states,
    Derivatives &gstates,
    float dt
)
{
    states.v += dt * gstates.v;
    states.n += dt * gstates.n;
    states.m += dt * gstates.m;
    states.h += dt * gstates.h;
}



#### NeuroDriver Code

In [10]:
nd_code_generator = NeuroDriverKernelGenerator(hh)
nd_code_generator.generate()
print(nd_code_generator.cuda_src)



#define  GNA		120.0
#define  GK		36.0
#define  GL		0.3
#define  ENA		50.0
#define  EK		-77.0
#define  EL		-54.387

#define  V_MIN		-80
#define  V_MAX		30
#define  N_MIN		0.0
#define  N_MAX		1.0
#define  M_MIN		0.0
#define  M_MAX		1.0
#define  H_MIN		0.0
#define  H_MAX		1.0

struct States {
    float v;
    float n;
    float m;
    float h;
};

struct Derivatives {
    float v;
    float n;
    float m;
    float h;
};


__device__ void clip(States &states)
{
    states.v = fmaxf(states.v, V_MIN);
    states.v = fminf(states.v, V_MAX);
    states.n = fmaxf(states.n, N_MIN);
    states.n = fminf(states.n, N_MAX);
    states.m = fmaxf(states.m, M_MIN);
    states.m = fminf(states.m, M_MAX);
    states.h = fmaxf(states.h, H_MIN);
    states.h = fminf(states.h, H_MAX);
}

__device__ void forward(
    States &states,
    Derivatives &gstates,
    float dt
)
{
    states.v += dt * gstates.v;
    states.n += dt * gstates.n;
    states.m += dt * gstates.m;
    states.h += dt * gstates.h;
}



## Alpha Synapse

In [4]:
alpha = Alpha()
print(inspect.getsource(hh.ode))

    def ode(self, stimulus=0.):

        alpha = np.exp(-(self.v+55.)/10.)-1.
        beta = (0.125*np.exp(-(self.v+65.)/80.))
        if abs(alpha) <= 1e-7:
            self.d_n = 0.1 * (1.-self.n) - beta * self.n
        else:
            self.d_n = (-0.01*(self.v+55.)/alpha) * (1.-self.n) - beta * self.n

        alpha = np.exp(-(self.v+40.)/10.)-1.
        beta = (4.*np.exp(-(self.v+65.)/18.))
        if abs(alpha) <= 1e-7:
            self.d_m = (1.-self.m) - beta * self.m
        else:
            self.d_m = (-0.1*(self.v+40.)/alpha) * (1.-self.m) - beta * self.m

        alpha = (0.07 * np.exp(-(self.v+65.)/20.))
        beta = 1. / (np.exp(-(self.v+35.)/10.)+1.)
        self.d_h = alpha * (1-self.h) - beta * self.h

        i_na = self.gNa * np.power(self.m, 3) * self.h * (self.v - self.ENa)
        i_k = self.gK * np.power(self.n, 4) * (self.v - self.EK)
        i_l = self.gL * (self.v - self.EL)

        self.d_v = stimulus - i_na - i_k - i_l



#### CUDA Code

In [5]:
code_generator = CudaKernelGenerator(alpha)
code_generator.generate()
print(code_generator.cuda_src)



#define  AR		12.5
#define  AD		12.19
#define  GMAX		1.0


struct States {
    float s;
    float u;
};

struct Derivatives {
    float s;
    float u;
};



__device__ void forward(
    States &states,
    Derivatives &gstates,
    float dt
)
{
    states.s += dt * gstates.s;
    states.u += dt * gstates.u;
}

__device__ int ode(
    States &states,
    Derivatives &gstates,
    float &stimulus
)
{
    float tmp;

    gstates.s = states.u;
    tmp = (AR * AD);
    gstates.u = (((-(AR + AD)) * states.u) - (tmp * states.s));
    if (stimulus) {
        states.u = (states.u + tmp);
    }
    return 0;
}



__global__ void Alpha (
    int num_thread,
    float dt,
    float *g_s,
    float *g_u,
    float *g_stimulus
)
{
    /* TODO: option for 1-D or 2-D */
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int total_threads = gridDim.x * blockDim.x;

    for (int nid = tid; nid < num_thread; nid += total_threads) {

        States states;
        Derivatives gstates;

        /* 

## Oizumi Model

In [6]:
class Oizumi(Model):
    Default_Inters = {'p': 0., 'NT':0}
    Default_States = {'N': (51., 0., 51.), 'g': (0., 0., 1000.)}
    Default_Params = {'N0': 51, 'q': 1.07, 'tauO': 10, 'tauN0': 100, 'pmax': 0.79, 'Kpre': 0.0035}
    
    def ode(self, input=0, f=1.):
        """
        Arguments:
            input (bool): spike indicator.
            f (float): overall spike rate.
        """
        self.p = self.pmax * np.exp(-self.Kpre * f)
        self.d_N = (self.N0 - self.N) ** 2 / self.tauN0
        self.d_g = -self.g / self.tauO
        
        if input == 1:
            self.NT = self.N*self.p
            self.g += self.NT*self.q
            self.N -= self.NT
        else:
            self.NT = 0

    def get_conductance(self):
        return self.g
    
oz = Oizumi()
print(inspect.getsource(oz.ode))

    def ode(self, input=0, f=1.):
        """
        Arguments:
            input (bool): spike indicator.
            f (float): overall spike rate.
        """
        self.p = self.pmax * np.exp(-self.Kpre * f)
        self.d_N = (self.N0 - self.N) ** 2 / self.tauN0
        self.d_g = -self.g / self.tauO
        
        if input == 1:
            self.NT = self.N*self.p
            self.g += self.NT*self.q
            self.N -= self.NT
        else:
            self.NT = 0



#### CUDA Code

In [7]:
code_generator = CudaKernelGenerator(oz)
code_generator.generate()
print(code_generator.cuda_src)



#define  N0		51
#define  Q		1.07
#define  TAUO		10
#define  TAUN0		100
#define  PMAX		0.79
#define  KPRE		0.0035

#define  N_MIN		0.0
#define  N_MAX		51.0
#define  G_MIN		0.0
#define  G_MAX		1000.0

struct States {
    float N;
    float g;
};

struct Derivatives {
    float N;
    float g;
};


__device__ void clip(States &states)
{
    states.N = fmaxf(states.N, N_MIN);
    states.N = fminf(states.N, N_MAX);
    states.g = fmaxf(states.g, G_MIN);
    states.g = fminf(states.g, G_MAX);
}

__device__ void forward(
    States &states,
    Derivatives &gstates,
    float dt
)
{
    states.N += dt * gstates.N;
    states.g += dt * gstates.g;
}

__device__ int ode(
    States &states,
    Derivatives &gstates,
    float &input,
    float &f
)
{

    self = (PMAX * expf(((-KPRE) * f)));
    gstates.N = (powf((N0 - states.N), 2) / TAUN0);
    gstates.g = ((-states.g) / TAUO);
    if ((input == 1)) {
        self = (states.N * self);
        states.g = (states.g + (self * Q));
        state

In [8]:
import numpy as np
class OTP(Model):
    Default_States = dict(
        v=0.,
        I=0.,
        uh=(0., 0., 50000.),
        duh=0.,
        x1=(0., 0., 1.),
        x2=(0., 0., 1.),
        x3=(0., 0., 1000.))
    Default_Params = dict(
        bf=1.0,
        s1=1.,
        s2=0.135,
        d1=13.,
        c1=13.*0.00145,
        a2=128.77,
        b2=87.89,
        a3=2.100,
        b3=1.200,
        k23=6089,
        nCR=1.,
        CCR=0.075,
        ICR=117.74,
        L=0.8,
        W=45.)
    
    def ode(self, stimulus=0.):
        self.d_uh = self.duh
        self.d_duh = -2*self.W*self.L*self.duh + self.W*self.W*(stimulus-self.uh)
        self.v = self.uh + self.s2*self.duh
        self.v = (self.v > 0) * self.v

        self.d_x1 = self.c1*self.bf*self.v*(1.-self.x1) - self.d1*self.x1
        f = np.cbrt(self.x2*self.x2) * np.cbrt(self.x3*self.x3)
        self.d_x2 = self.a2*self.x1*(1.-self.x2) - self.b2*self.x2 - self.k23*f
        self.d_x3 = self.a3*self.x2 - self.b3*self.x3

        self.I = self.ICR * self.x2 / (self.x2 + self.CCR)

In [9]:
otp = OTP()
code_generator = CudaKernelGenerator(otp)
code_generator.generate()
print(code_generator.cuda_src)



#define  BF		1.0
#define  S1		1.0
#define  S2		0.135
#define  D1		13.0
#define  C1		0.01885
#define  A2		128.77
#define  B2		87.89
#define  A3		2.1
#define  B3		1.2
#define  K23		6089
#define  NCR		1.0
#define  CCR		0.075
#define  ICR		117.74
#define  L		0.8
#define  W		45.0

#define  UH_MIN		0.0
#define  UH_MAX		50000.0
#define  X1_MIN		0.0
#define  X1_MAX		1.0
#define  X2_MIN		0.0
#define  X2_MAX		1.0
#define  X3_MIN		0.0
#define  X3_MAX		1000.0

struct States {
    float v;
    float I;
    float uh;
    float duh;
    float x1;
    float x2;
    float x3;
};

struct Derivatives {
    float uh;
    float duh;
    float x1;
    float x2;
    float x3;
};


__device__ void clip(States &states)
{
    states.uh = fmaxf(states.uh, UH_MIN);
    states.uh = fminf(states.uh, UH_MAX);
    states.x1 = fmaxf(states.x1, X1_MIN);
    states.x1 = fminf(states.x1, X1_MAX);
    states.x2 = fmaxf(states.x2, X2_MIN);
    states.x2 = fminf(states.x2, X2_MAX);
    states.x3 = fmaxf(states.x3, X3_MIN)